In [2]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

In [3]:
df = pd.read_csv("src/data/fooddataset.csv", encoding='latin-1')
df.head()

,Unnamed: 0,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1961,1961,1000 persons,8954.0,NaN
1,1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1962,1962,1000 persons,9142.0,NaN
2,2,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1963,1963,1000 persons,9340.0,NaN
3,3,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1964,1964,1000 persons,9547.0,NaN
4,4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1965,1965,1000 persons,9765.0,NaN


In [4]:
df = pd.read_csv("src/data/FoodBalanceSheetsHistoric_E_All_Data.csv", encoding='latin-1')
df.head()


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,8954.00,NaN,9142.00,...,27708.00,NaN,28398.00,NaN,29105.00,NaN,29825.00,NaN,30552.00,NaN
1,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,Fc,2917.00,...,2081.00,Fc,2104.00,Fc,2107.00,Fc,2100.00,Fc,2090.00,Fc
2,2,Afghanistan,2901,Grand Total,674,Protein supply quantity (g/capita/day),g/capita/day,84.91,Fc,82.98,...,57.79,Fc,58.14,Fc,58.91,Fc,58.91,Fc,58.25,Fc
3,2,Afghanistan,2901,Grand Total,684,Fat supply quantity (g/capita/day),g/capita/day,37.51,Fc,37.61,...,30.72,Fc,33.88,Fc,33.08,Fc,33.37,Fc,33.52,Fc
4,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,Fc,2672.00,...,1871.00,Fc,1888.00,Fc,1891.00,Fc,1883.00,Fc,1873.00,Fc


In [4]:
df = df.pivot_table(values = 'Value', columns = 'Year', index = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element']).reset_index()

In [52]:
mozambique = df[df.Area == 'Mozambique']

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
# grouped_mo = grouped_mo.iloc[[5,19,20]]
grouped_mo
# grouped_mo.iloc[18]
# melted_mo = pd.melt(grouped_mo.iloc[18:19], id_vars=['Element'], value_vars=grouped_mo.columns)
# melted_mo['variable'] = melted_mo.variable.str.lstrip('Y')

Year
Element    Prop_Food
1961        0.920674
1962        0.932394
1963        0.901919
1964        0.914861
1965        0.913651
1966        0.901053
1967        0.905728
1968        0.879272
1969        0.899663
1970        0.895092
1971        0.932578
1972        0.904551
1973          0.9385
1974        0.905725
1975        0.875571
1976         0.88231
1977        0.863275
1978        0.857288
1979        0.850434
1980        0.860334
1981        0.833292
1982        0.810669
1983        0.794675
1984        0.810593
1985        0.813349
1986        0.782302
1987         0.80416
1988        0.797401
1989        0.783765
1990        0.835386
1991        0.773478
1992        0.732745
1993        0.731883
1994        0.725261
1995        0.769099
1996         0.81765
1997        0.835477
1998        0.844324
1999        0.860551
2000        0.841017
2001        0.852836
2002        0.867242
2003        0.909684
2004        0.888109
2005        0.836135
2006        0.870283
2007    

In [4]:
import json

f = open("src/data/world3.json")
geojson = json.load(f)


In [10]:
df[(df.Area == "France") & (df.Element == "Food")]

# df.Element

AttributeError: 'DataFrame' object has no attribute 'ELement'

In [5]:
# Replace country names to make them map with the geojson
area_replacements = [
  ("United Kingdom of Great Britain and Northern Ireland", "United Kingdom"), 
  ("United States of America", "United States"),
  ("Czechia", "Czech Republic"),
  ("Republic of Moldova", "Moldova"),
  ("North Macedonia", "Macedonia"),
  ("Iran (Islamic Republic of)", "Iran"),
  ("Congo", "Democratic Republic of Congo"),
  ("Venezuela (Bolivarian Republic of)", "Venezuela"),
  ("Bolivia (Plurinational State of)", "Bolivia"),
  ("Viet Nam", "Vietnam")
]

for pair in area_replacements:
  df.Area = df.Area.replace(pair[0], pair[1])

In [25]:
# Create the output geojson
a = df[(df.Item == "Grand Total") & (df.Element == "Food supply (kcal/capita/day)")]
common_items = df.Item.value_counts().head(10).index

for feature in geojson['features']:
  try: 
    # Compute feed, food, kcal/habitants
    area = feature['properties']['NAME_LONG']
    mozambique = df[df.Area == area]

    grouped_mo = mozambique.groupby('Element').sum()
    grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
    grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()

    ids = [3, 4, 18, 19]
    labels = ["feed", "food", "prop_food", "prop_feed"]

    for id, label in enumerate(labels):
      try:
        feature['properties'][label] = list(grouped_mo.iloc[ids[id]][1:] * 1000)
      except IndexError:
        print("INDEX ERROR: ", area, grouped_mo)

    feature['properties']['food_supply'] = list(a[a.Area == area].groupby("Element").sum().values[0][3:])

    # Compute amount of food per most common types of food
    df2 = mozambique.groupby(['Item','Element']).sum().drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
    df2 = df2[df2.Item.isin(common_items)]
    df2 = df2[df2.Element.isin(["Feed", "Food"])]

    out = []
    items_processed = []

    for item in df2.Item.unique():
      if not item in items_processed:
        items_processed.append(item)
      else:
        continue

      feed = df2[(df2.Item == item) & (df2.Element == "Feed")].reset_index().drop(columns=["Item", "Element", "index"])
      food = df2[(df2.Item == item) & (df2.Element == "Food")].reset_index().drop(columns=["Item", "Element", "index"])
      combined = (food + feed).fillna(1)

      feed = (feed / combined * 100).fillna(0).clip(upper=100).values
      food = (food / combined * 100).fillna(0).clip(upper=100).values

      if (len(feed) == 0): feed = [[]]
      feed = list(feed[0])
      

      if (len(food) == 0): food = [[]]
      food = list(food[0])

      out.append({ "group": item, "feed": feed, "food": food  })

    feature['properties']["types_of_food"] = out
  
  except KeyError:
    print("Country not found: ", area)
  
  

Processing Cereals - Excluding Beer
Processing Eggs
Processing Maize and products
Processing Milk - Excluding Butter
Processing Miscellaneous
Processing Oilcrops
Processing Potatoes and products
Processing Pulses
Processing Starchy Roots
Processing Wheat and products
Processing Cereals - Excluding Beer
Processing Eggs
Processing Maize and products
Processing Milk - Excluding Butter
Processing Miscellaneous
Processing Oilcrops
Processing Potatoes and products
Processing Pulses
Processing Starchy Roots
Processing Wheat and products
Processing Cereals - Excluding Beer
Processing Eggs
Processing Maize and products
Processing Milk - Excluding Butter
Processing Miscellaneous
Processing Oilcrops
Processing Potatoes and products
Processing Pulses
Processing Starchy Roots
Processing Wheat and products
Country not found:  Aland Islands
Country not found:  Andorra
Processing Cereals - Excluding Beer
Processing Eggs
Processing Maize and products
Processing Milk - Excluding Butter
Processing Miscel

In [27]:
out = open("src/data/out.json", "w+")
json.dump(geojson, out)


In [35]:

mozambique = df[df.Area == "Mozambique"]

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1)
# grouped_mo = grouped_mo.iloc[[3, 45,17,18]]
list(grouped_mo.loc['Prop_Food'])
# list(grouped_mo.iloc[3])

[0.920674358974359,
 0.9323944723618092,
 0.9019187692307694,
 0.9148607930577317,
 0.9136511682794429,
 0.9010534827307382,
 0.9057284955352096,
 0.879272365365476,
 0.8996625202812331,
 0.8950915053422194,
 0.9325781088350922,
 0.9045512496205605,
 0.9385004945598417,
 0.9057253235464556,
 0.8755706214689267,
 0.8823102310231025,
 0.8632747703558371,
 0.8572881061854775,
 0.8504341216216216,
 0.8603343765442268,
 0.8332920652785595,
 0.8106685521250687,
 0.7946748249057185,
 0.8105931368835693,
 0.8133488267486318,
 0.7823015458143528,
 0.8041600000000001,
 0.7974005543486403,
 0.7837648376259799,
 0.8353862028301887,
 0.7734777601842524,
 0.7327449477351917,
 0.7318827637144968,
 0.7252611143467287,
 0.7690989863596547,
 0.8176504889752779,
 0.8354773869346735,
 0.8443242464411274,
 0.8605506236655805,
 0.8410171810024075,
 0.8528361545832003,
 0.8672422360248446,
 0.9096844965531122,
 0.8881090072899845,
 0.83613469776297,
 0.8702830407189512,
 0.8507144467998736,
 0.89098141721214

In [30]:
mozambique = df[df.Area == "Canada"]

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()

ids = [3, 4, 17, 18]
labels = ["feed", "food", "prop_food", "prop_feed"]
list(grouped_mo.iloc[3][1:] * 1000)
# for idx, label in enumerate(labels):
#   print(list(grouped_mo.iloc[idx][1:]))

# feature['properties']['food_supply'] = list(a[a.Area == area].groupby("Element").sum().values[0][3:])


[22888580.000000004,
 28151860.000000004,
 29683580.0,
 28624140.0,
 30452239.999999996,
 33332460.0,
 31282160.000000004,
 31848640.0,
 39126940.0,
 39205280.00000001,
 41240979.99999999,
 34691240.00000001,
 36159499.99999999,
 34715079.99999999,
 37095380.00000001,
 33210940.000000004,
 36261280.0,
 35748959.99999999,
 37458920.0,
 36696150.0,
 38841409.99999999,
 36170940.0,
 37519469.99999999,
 32656200.0,
 40393210.0,
 40172969.99999999,
 41983649.99999999,
 37819390.0,
 39667070.00000001,
 40346549.99999999,
 37995619.99999999,
 37172890.0,
 49459169.99999999,
 46512990.0,
 47211039.99999999,
 49387190.0,
 48106190.0,
 52725499.99999999,
 51259540.0,
 49803150.00000001,
 48126010.0,
 47438060.0,
 55044649.99999999,
 49719470.0,
 50373979.99999999,
 45315610.0,
 49737229.99999999,
 48320800.0,
 40602450.0,
 37079289.99999999,
 38177770.0,
 41687350.00000001,
 52272650.0]